## Import Libs

In [1]:
import os
import random
import gc
import time
import copy
import sys
import numpy as np
import pandas as pd
import h5py
from io import BytesIO

from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn.functional as F
from torch import nn
import timm

from PIL import Image
import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm import tqdm
from collections import defaultdict

import warnings # 避免一些可以忽略的报错
warnings.filterwarnings('ignore')
# For colored terminal text
from colorama import Fore, Back, Style
b_ = Fore.BLUE
sr_ = Style.RESET_ALL

## CONFIG

In [2]:
is_debug = False

class CONFIG:
    seed = 308
    
    # 256 tiny_vit ---------- 3h+
    # 512 efficientnet convnext -------------- 17mins+
    #     efficientvit_b0 --------------- 7mins+
    test_batch_size = 512
    img_size = [160, 160]
    n_classes = 1
    n_folds = 5
    
    n_accumulate = 1.0
    n_workers = os.cpu_count()

    DataParallel = True
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    """
    tf_efficientnet_b0_ns
    tiny_vit_21m_512.dist_in22k_ft_in1k
    eva02_base_patch14_224.mim_in22k
    convnext_tiny.fb_in22k_ft_in1k_384
    tf_efficientnetv2_s.in21k_ft_in1k
    efficientvit_b0.r224_in1k
    
    edgenext_base.in21k_ft_in1k
    """
    model_name = "tf_efficientnetv2_s.in21k_ft_in1k"
    ckpt_path = "/kaggle/input/20240827-efficientnetv2s-train1954-cv15496"
    use_gempool = False

    test_csv = "/kaggle/input/isic-2024-challenge/test-metadata.csv"
    test_img_hdf5 = "/kaggle/input/isic-2024-challenge/test-image.hdf5"
    
#     # debug
#     test_csv = "/kaggle/input/isic-2024-challenge/train-metadata.csv"
#     test_img_hdf5 = "/kaggle/input/isic-2024-challenge/train-image.hdf5"

if CONFIG.DataParallel:
    os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'
    print("IN DataParallel!")
else:
    os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
    print("NO IN DataParallel!")

IN DataParallel!


## Set Random Seed

In [3]:
def set_seed(seed=308):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
set_seed(CONFIG.seed)

## Data Progress

In [4]:
test = pd.read_csv(CONFIG.test_csv)
test

,isic_id,patient_id,age_approx,sex,anatom_site_general,clin_size_long_diam_mm,image_type,tbp_tile_type,tbp_lv_A,tbp_lv_Aext,...,tbp_lv_radial_color_std_max,tbp_lv_stdL,tbp_lv_stdLExt,tbp_lv_symm_2axis,tbp_lv_symm_2axis_angle,tbp_lv_x,tbp_lv_y,tbp_lv_z,attribution,copyright_license
0,ISIC_0015657,IP_6074337,45.0,male,posterior torso,2.70,TBP tile: close-up,3D: XP,22.80433,20.007270,...,0.304827,1.281532,2.299935,0.479339,20,-155.06510,1511.222000,113.980100,Memorial Sloan Kettering Cancer Center,CC-BY
1,ISIC_0015729,IP_1664139,35.0,female,lower extremity,2.52,TBP tile: close-up,3D: XP,16.64867,9.657964,...,0.000000,1.271940,2.011223,0.426230,25,-112.36924,629.535889,-15.019287,"Frazer Institute, The University of Queensland...",CC-BY
2,ISIC_0015740,IP_7142616,65.0,male,posterior torso,3.16,TBP tile: close-up,3D: XP,24.25384,19.937380,...,0.230742,1.080308,2.705857,0.366071,110,-84.29282,1303.978000,-28.576050,FNQH Cairns,CC-BY


## Dataset and DataLoader

In [5]:
def transform(img):
    composition = A.Compose([
        A.Resize(CONFIG.img_size[0], CONFIG.img_size[1]),
        A.Normalize(),
        ToTensorV2(),
    ])
    return composition(image=img)["image"]

In [6]:
class MyDataset(Dataset):
    def __init__(self, df, transform=None):
        super().__init__()
        self.df = df
        self.fp_hdf = h5py.File(CONFIG.test_img_hdf5, mode="r")
        self.transform = transform

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx, :]
        img_id = row.isic_id
        label = str(img_id)
        
        img = np.array( Image.open(BytesIO(self.fp_hdf[img_id][()])) )
        img = np.array(img).astype(np.float32)

        if self.transform != None:
            img = self.transform(img)

        return img, label

In [7]:
def prepare_loaders(df):
    
    test_datasets = MyDataset(df=df, transform=transform)
    
    test_loader = DataLoader(test_datasets, batch_size=CONFIG.test_batch_size, num_workers=CONFIG.n_workers, shuffle=False, pin_memory=True)
    
    return test_loader

In [8]:
# test_loader = prepare_loaders(test)
# x, y = next(iter(test_loader))
# x.shape

## DL Model

In [ ]:
def updata_req_grad(models, requires_grad=True):
    for model in models:
        for param in model.parameters():
            param.requires_grad = requires_grad

In [ ]:
class GeMPool(nn.Module):
    def __init__(self, p=3, eps=1e-6):
        super(GeMPool, self).__init__()
        self.p = nn.Parameter(torch.ones(1) * p)
        self.eps = eps

    def forward(self, x):
        return self.gem(x, p=self.p, eps=self.eps)
    
    def gem(self, x, p=3, eps=1e-6):
        return torch.mean(x.clamp(min=eps).pow(p), dim=(-2, -1)).pow(1./p)
    
    def __repr__(self):
        return self.__class__.__name__ + f'(p={self.p.data.tolist()[0]:.4f}, eps={self.eps})'

In [ ]:
class ISIC2024Model(nn.Module):
    def __init__(self):
        super(ISIC2024Model, self).__init__()
        self.backbone = timm.create_model(model_name=CONFIG.model_name, 
                                          pretrained=False)
        
        if "efficientnet" in CONFIG.model_name:
            in_features = self.backbone.classifier.in_features
            self.backbone.classifier = nn.Identity()
            if CONFIG.use_gempool:
                self.backbone.global_pool = GeMPool()
        elif "convnext" in CONFIG.model_name or "tiny_vit" in CONFIG.model_name:
            in_features = self.backbone.head.fc.in_features
            self.backbone.head.fc = nn.Identity()
        elif "eva" in CONFIG.model_name:
            in_features = 768
        elif "efficientvit" in CONFIG.model_name:
            in_features = self.backbone.head.classifier[4].in_features
            self.backbone.head.classifier[4] = nn.Identity()
        elif "edgenext" in CONFIG.model_name:
            in_features = self.backbone.head.fc.in_features
            self.backbone.head.fc = nn.Identity()

        self.head = nn.Sequential(
            nn.Linear(in_features, CONFIG.n_classes)
        )
        
    def forward(self, x):
        _tmp = self.backbone(x)
        output = self.head(_tmp)
        return output

In [ ]:
model = ISIC2024Model()
model

## Load Models

In [ ]:
models = []

paths = os.listdir(CONFIG.ckpt_path)
# paths = ["1_CV0.1547_Loss0.6784_epoch2.bin",
#          "2_CV0.1708_Loss0.5088_epoch5.bin",
#          "3_CV0.1791_Loss0.3739_epoch7.bin",
#          "4_CV0.1654_Loss0.5382_epoch3.bin",
#          "5_CV0.1812_Loss0.4236_epoch8.bin"]

if CONFIG.DataParallel:
    device_ids = [0, 1]
    for i in range(CONFIG.n_folds):
        model = ISIC2024Model()
        model = torch.nn.DataParallel(model, device_ids=device_ids)
        model = model.cuda()
        model.load_state_dict(torch.load(os.path.join(CONFIG.ckpt_path, paths[i])))
        model.eval()
        models.append(model)
else:
    for i in range(CONFIG.n_folds):
        model = ISIC2024Model()
        model = model.cuda()
        model.load_state_dict(torch.load(os.path.join(CONFIG.ckpt_path, paths[i])))
        model.eval()
        models.append(model)

## Infer Function

In [ ]:
def Infer(models, test_loader):
    y_preds = []
    bar = tqdm(enumerate(test_loader), total=len(test_loader))
    with torch.no_grad():
        for step, (images, study_id) in bar:
            
            if CONFIG.DataParallel:
                images = images.cuda().float()
            else:
                images = images.to(CONFIG.device, dtype=torch.float)
                
            outputs = 0
                
            for model in models:
                output = model(images)
                outputs += output
            outputs = outputs / len(models)
            outputs = F.sigmoid(outputs)
            y_preds.append(outputs.detach().flatten().cpu().numpy())
            
    y_preds = np.concatenate(y_preds)
    return y_preds

## Start Infer

In [ ]:
test_loader = prepare_loaders(test)

preds = Infer(models, test_loader)

## Make Submission

In [ ]:
sub = pd.DataFrame()
sub['isic_id'] = test["isic_id"]
sub['target'] = preds
sub.head(25)

In [ ]:
sub.to_csv('submission.csv', index=False)
pd.read_csv('submission.csv').head()